# Actividad 5: Agregaciones a Dataset

**Asignatura:** Big Data Aplicado.
**Estudiante:** Byron V. Blatch Rodríguez.
**Fecha:** 26/11/2025

## Objetivos de la actividad
1. Generar agregaciones intuyendo qué información podemos adelantar o puede ser frecuentemente consultada, optimizándola.
2. Utilizar herramientas de base de datos como MongoDB y de virtualización como Docker.
---

En esta ocasión simularé la subida de mis CSV en una base de datos, MongoDB será la herramienta que utilizaré. No haré una instalación nativa de mongo, sino que utilizaré la herramienta Docker para ejecutar MongoDB virtualizadamente en mi equipo, permitiéndome hacer la simulación.

Para ello, en mi Pop OS! 22.04 he instalado Docker, y he descargado la imagen con el comando `docker run -d --name mi-mongo-local -p 27777:27017 mongo:latest`. Me aseguro qu ese esté ejecutando con `docker ps` y finalmente procedo a codificar la subida.

In [10]:
# Importo las librería para la actividad
from pymongo import MongoClient # Librería para conectar con MongoDB
import pandas as pd           # Librería para manejo de dataframes
import numpy as np            # Librería para manejo de arrays y operaciones matemáticas
import matplotlib.pyplot as plt # Librería para graficar


# Conexión a mi Mongo de Docker
client = MongoClient("mongodb://localhost:27777") # Puerto mapeado en docker run [web:80] [web:62]

db = client["bigdata"]        # Nombre de la database
clientes_col = db["clientes"] # Colección de clientes
consumos_col = db["consumos"] # Colección consumos

print("Conexión establecida correctamente")

Conexión establecida correctamente


In [11]:
# Ahora cargamos nuestros CSV a la base de datos

df_prop = pd.read_csv("data/propietarios.csv")
df_cons = pd.read_csv("data/consumos_generacion.csv")

# Convertir la columna 'timestamp' a datetime antes de la inserción
df_cons['timestamp'] = pd.to_datetime(df_cons['timestamp'])

print(df_prop.head())
print(df_cons.head())

                              client_id                      nombre  \
0  99dfec6b-5bcc-453f-a0b9-aa1407614605      Hilda Bejarano-Anglada   
1  9f5d6a32-f462-462b-a2ff-5e313b1ff8a6  Baudelio Ferrández Naranjo   
2  805d9c84-7359-46ca-a59c-1c708b96d6f5  Juan Bautista Mármol Arnau   
3  9ca72c79-f619-4ccd-9c8e-f7d2935aef2c             Eutimio Company   
4  a3cb15e4-fb47-4f47-8489-68539e88cd51            Ale Calleja Uría   

                                           direccion      ciudad  \
0       Calle de Marcial Molins 78, La Coruña, 05578   Tarragona   
1  Avenida de Lisandro González 6 Puerta 3 , Sori...      Ciudad   
2              Rambla Lino Calleja 40, Teruel, 17484      Ciudad   
3       Pasaje de Duilio Vila 376, Valladolid, 32478  Pontevedra   
4  Paseo de María Fernanda Cañellas 6 Apt. 75 , T...      Ciudad   

                        email  
0          freina@example.com  
1  escalonasabina@example.org  
2        nhurtado@example.org  
3         luisa29@example.org  
4   

In [12]:
# Limpiar colecciones por si ya tenían datos de pruebas
clientes_col.delete_many({})
consumos_col.delete_many({})

# 2) Insertar propietarios
docs_prop = df_prop.to_dict("records")
clientes_col.insert_many(docs_prop)
print(f"Insertados {len(docs_prop)} propietarios en 'clientes'.")

# 3) Insertar consumos (tomará tiempo ya que son millones)
docs_cons = df_cons.to_dict("records")
consumos_col.insert_many(docs_cons)
print(f"Insertados {len(docs_cons)} registros en 'consumos'.")

Insertados 500 propietarios en 'clientes'.
Insertados 4380000 registros en 'consumos'.


Mis datos ya se encuentran subidos a MongoDB ejecutado virtualmente en local gracias a Docker. Lo siguiente que haré es observar gráficos de mis datos sintéticos antes de establecer cuáles podrían ser los patrones que me ayudarán a detectar "sospechosos" en la red eléctrica. Planteamos un pipeline exploratorio.

In [ ]:
# Por comodidad trabajaremos con agregaciones en MongoDB para analizar los datos.

# Creamos pipelines sin filtros de umbrales para observarlo todo.
"""
pipeline_exploratorio = [
    # Paso 1: Extraer día desde timestamp
    {
        "$project": {
            "client_id": 1,
            "consumption_kwh": 1,
            "day": {
                "$dateToString": {
                    "format": "%Y-%m-%d",
                    "date": "$timestamp"
                }
            }
        }
    },
    # Paso 2: Agrupar por cliente y día
    {
        "$group": {
            "_id": {
                "client_id": "$client_id",
                "day": "$day"
            },
            "avg_consumption": {"$avg": "$consumption_kwh"},
            "std_consumption": {"$stdDevSamp": "$consumption_kwh"},
            "count_hours": {"$sum": 1}
        }
    },
    # Paso 3: Solo días completos (24 horas)
    {
        "$match": {
            "count_hours": 24
        }
    }
]
"""

pipeline_exploratorio = [
    {
        "$project": {
            "client_id": 1,
            "consumo_kwh": 1,
            "day": {
                "$dateToString": {
                    "format": "%Y-%m-%d",
                    "date": "$timestamp"
                }
            }
        }
    },
    {
        "$group": {
            "_id": {
                "client_id": "$client_id",
                "day": "$day"
            },
            "avg_consumption": {"$avg": "$consumo_kwh"},
            "std_consumption": {"$stdDevSamp": "$consumo_kwh"},
            "count_hours": {"$sum": 1}
        }
    },
    {
        "$match": {
            "count_hours": 24
        }
    }
]

print("Ejecutando agregación...")
todos_los_dias = list(consumos_col.aggregate(pipeline_exploratorio))
print(f"✓ Total de días completos: {len(todos_los_dias)}")

print("Ejecutando agregación sobre millones de registros...")
print("(Esto puede tardar 10-30 segundos)")

todos_los_dias = list(consumos_col.aggregate(pipeline_exploratorio))

print(f"\n✓ Agregación completada")
print(f"Total de días completos analizados: {len(todos_los_dias)}")

Ejecutando agregación...
✓ Total de días completos: 182500
Ejecutando agregación sobre millones de registros...
(Esto puede tardar 10-30 segundos)

✓ Agregación completada
Total de días completos analizados: 182500


In [14]:
# Ahora extraemos los datos exploratorios para generar la gráfica.

datos = []

for doc in todos_los_dias:
    datos.append({
        'client_id': doc['_id']['client_id'],
        'day': doc['_id']['day'],
        'avg_consumption': doc['avg_consumption'],
        'std_consumption': doc['std_consumption']
    })

df_analisis = pd.DataFrame(datos)

print("✓ Datos convertidos a Dataframe")
print(f"\nPrimeras 5 filas:")
print(df_analisis.head())
print(f"Dimensiones: {df_analisis.shape[0]} filas y {df_analisis.shape[1]} columnas")

✓ Datos convertidos a Dataframe

Primeras 5 filas:
                              client_id         day  avg_consumption  \
0  f0f46bf8-3dfc-4ad9-b29a-94a51d63cccf  2025-02-05         8.556667   
1  9a7dd9eb-5304-460f-be28-aaad29c5574b  2025-05-09        10.331667   
2  ba73fdd6-6c93-4731-b5e3-e7f5a3bc7cd1  2025-10-28         9.283750   
3  e1e79bae-1980-489b-9634-f4bda06625ee  2025-05-09        11.532500   
4  43a32471-f82d-454e-b991-efa01699443a  2025-06-27        12.098333   

   std_consumption  
0         6.022116  
1         5.631001  
2         6.261559  
3         5.909257  
4         5.131773  
Dimensiones: 182500 filas y 4 columnas


In [15]:
print("="*70)
print("RESUMEN ESTADÍSTICO")
print("="*70)
print(df_analisis[['avg_consumption', 'std_consumption']].describe())

# Calcular percentiles clave
print("\n" + "="*70)
print("Umbrales que usaré")
print("="*70)

# Top 10% de consumo
umbral_consumo = df_analisis['avg_consumption'].quantile(0.99)
print(f"\nTop 10% de consumo medio: >= {umbral_consumo:.2f} kWh/hora")

# Bottom 10% de variación (más plano)
umbral_std = df_analisis['std_consumption'].quantile(0.01)
print(f"Bottom 10% de variación: <= {umbral_std:.2f} kWh")

print(f"\n💡 Usaremos estos valores como umbrales para detectar sospechosos.")

RESUMEN ESTADÍSTICO
       avg_consumption  std_consumption
count    182500.000000    182500.000000
mean         10.007187         5.744621
std           1.177541         0.559205
min           4.872917         3.164632
25%           9.207500         5.375465
50%          10.013958         5.758746
75%          10.805833         6.130014
max          15.170417         7.899445

Umbrales que usaré

Top 10% de consumo medio: >= 12.72 kWh/hora
Bottom 10% de variación: <= 4.38 kWh

💡 Usaremos estos valores como umbrales para detectar sospechosos.


Podemos observar que hemos obtenido un total de 182.500 registros en cuanto a consumos diarios se refiere. Aunque tengamos los números podemos demostrarlos de manera más gráfica y agradable para el ojo humano y de estudio. Por tanto procederemo a graficar.